# Read DWD CDC Cloudiness_recent
Time Series

The main idea behind this activity is to see if there is a correlation between cloud coverage and sales of the stores.

In [189]:
import datetime as dt

## FTP Connection

### Connection Parameters

In [190]:
server = "opendata.dwd.de"
user   = "anonymous"
passwd = ""

### FTP Directory Definition and Station Description Filename Pattern

In [303]:
# The topic of interest.
topic_dir = "hourly/cloudiness/recent/"

# This is the search pattern common to ALL station description file names 
station_desc_pattern = "_Beschreibung_Stationen.txt"

# Below this directory tree node all climate data are stored.
ftp_climate_data_dir = "/climate_environment/CDC/observations_germany/climate/"
ftp_dir =  ftp_climate_data_dir + topic_dir

### Local Directories

In [304]:
local_ftp_dir         = "data/original/DWD/"      # Local directory to store local ftp data copies, the local data source or input data. 
local_ftp_station_dir = local_ftp_dir + topic_dir # Local directory where local station info is located
local_ftp_ts_dir      = local_ftp_dir + topic_dir # Local directory where time series downloaded from ftp are located

local_generated_dir   = "data/generated/DWD/" # The generated of derived data in contrast to local_ftp_dir
local_station_dir     = local_generated_dir + topic_dir # Derived station data, i.e. the CSV file
local_ts_merged_dir   = local_generated_dir + topic_dir # Parallel merged time series, wide data frame with one TS per column
local_ts_appended_dir = local_generated_dir + topic_dir # Serially appended time series, long data frame for QGIS TimeManager Plugin

In [305]:
print(local_ftp_dir)
print(local_ftp_station_dir)
print(local_ftp_ts_dir)
print()
print(local_generated_dir)
print(local_station_dir)
print(local_ts_merged_dir)
print(local_ts_appended_dir)

data/original/DWD/
data/original/DWD/hourly/cloudiness/recent/
data/original/DWD/hourly/cloudiness/recent/

data/generated/DWD/
data/generated/DWD/hourly/cloudiness/recent/
data/generated/DWD/hourly/cloudiness/recent/
data/generated/DWD/hourly/cloudiness/recent/


In [306]:
import os
os.makedirs(local_ftp_dir,exist_ok = True) # it does not complain if the dir already exists.
os.makedirs(local_ftp_station_dir,exist_ok = True)
os.makedirs(local_ftp_ts_dir,exist_ok = True)

os.makedirs(local_generated_dir,exist_ok = True)
os.makedirs(local_station_dir,exist_ok = True)
os.makedirs(local_ts_merged_dir,exist_ok = True)
os.makedirs(local_ts_appended_dir,exist_ok = True)

### FTP Connect

In [307]:
import ftplib
ftp = ftplib.FTP(server)
res = ftp.login(user=user, passwd = passwd)
print(res)

230 Login successful.


In [308]:
ret = ftp.cwd(".")

In [309]:
#ftp.quit()

### FTP Grab File Function

In [310]:
def grabFile(ftpfullname,localfullname):
    try:
        ret = ftp.cwd(".") # A dummy action to chack the connection and to provoke an exception if necessary.
        localfile = open(localfullname, 'wb')
        ftp.retrbinary('RETR ' + ftpfullname, localfile.write, 1024)
        localfile.close()
    
    except ftplib.error_perm:
        print("FTP ERROR. Operation not permitted. File not found?")

    except ftplib.error_temp:
        print("FTP ERROR. Timeout.")

    except ConnectionAbortedError:
        print("FTP ERROR. Connection aborted.")



### Generate Pandas Dataframe from FTP Directory Listing

In [311]:
import pandas as pd
import geopandas as gpd
import os

def gen_df_from_ftp_dir_listing(ftp, ftpdir):
    lines = []
    flist = []
    try:    
        res = ftp.retrlines("LIST "+ftpdir, lines.append)
    except:
        print("Error: ftp.retrlines() failed. ftp timeout? Reconnect!")
        return
        
    if len(lines) == 0:
        print("Error: ftp dir is empty")
        return
    
    for line in lines:
#        print(line)
        [ftype, fsize, fname] = [line[0:1], int(line[31:42]), line[56:]]
#        itemlist = [line[0:1], int(line[31:42]), line[56:]]
#        flist.append(itemlist)
        
        fext = os.path.splitext(fname)[-1]
        
        if fext == ".zip":
            station_id = int(fname.split("_")[2])
        else:
            station_id = -1 
        
        flist.append([station_id, fname, fext, fsize, ftype])
        
        

    df_ftpdir = pd.DataFrame(flist,columns=["station_id", "name", "ext", "size", "type"])
    return(df_ftpdir)

In [312]:
df_ftpdir = gen_df_from_ftp_dir_listing(ftp, ftp_dir)

In [313]:
df_ftpdir.tail(10)

,station_id,name,ext,size,type
196,7394,stundenwerte_N_07394_akt.zip,.zip,45441,-
197,7395,stundenwerte_N_07395_akt.zip,.zip,45428,-
198,7396,stundenwerte_N_07396_akt.zip,.zip,44281,-
199,7403,stundenwerte_N_07403_akt.zip,.zip,45651,-
200,7410,stundenwerte_N_07410_akt.zip,.zip,45490,-
201,7412,stundenwerte_N_07412_akt.zip,.zip,45155,-
202,13674,stundenwerte_N_13674_akt.zip,.zip,45531,-
203,15000,stundenwerte_N_15000_akt.zip,.zip,45836,-
204,15207,stundenwerte_N_15207_akt.zip,.zip,45546,-
205,15444,stundenwerte_N_15444_akt.zip,.zip,45564,-


### Dataframe with Temp Zip Files

In [314]:
#df_ftpdir["ext"]==".zip"
df_zips = df_ftpdir[df_ftpdir["ext"]==".zip"]
df_zips.set_index("station_id", inplace = True)
df_zips.tail(10)

,name,ext,size,type
station_id,,,,
7394,stundenwerte_N_07394_akt.zip,.zip,45441,-
7395,stundenwerte_N_07395_akt.zip,.zip,45428,-
7396,stundenwerte_N_07396_akt.zip,.zip,44281,-
7403,stundenwerte_N_07403_akt.zip,.zip,45651,-
7410,stundenwerte_N_07410_akt.zip,.zip,45490,-
7412,stundenwerte_N_07412_akt.zip,.zip,45155,-
13674,stundenwerte_N_13674_akt.zip,.zip,45531,-
15000,stundenwerte_N_15000_akt.zip,.zip,45836,-
15207,stundenwerte_N_15207_akt.zip,.zip,45546,-


### Download the Station Description File

In [315]:
station_fname = df_ftpdir[df_ftpdir['name'].str.contains(station_desc_pattern)]["name"].values[0]
print(station_fname)

# ALternative
#station_fname2 = df_ftpdir[df_ftpdir["name"].str.match("^.*Beschreibung_Stationen.*txt$")]["name"].values[0]
#print(station_fname2)

N_Stundenwerte_Beschreibung_Stationen.txt


In [316]:
print("grabFile: ")
print("From: " + ftp_dir + station_fname)
print("To:   " + local_ftp_station_dir + station_fname)
grabFile(ftp_dir + station_fname, local_ftp_station_dir + station_fname)

grabFile: 
From: /climate_environment/CDC/observations_germany/climate/hourly/cloudiness/recent/N_Stundenwerte_Beschreibung_Stationen.txt
To:   data/original/DWD/hourly/cloudiness/recent/N_Stundenwerte_Beschreibung_Stationen.txt


In [317]:
# extract column names. They are in German (de)
# We have to use codecs because of difficulties with character encoding (German Umlaute)
import codecs

def station_desc_txt_to_csv(txtfile, csvfile):
    file = codecs.open(txtfile,"r","utf8")#"utf8", iso8859_2
    r = file.readline()
    file.close()
    colnames_de = r.split()
    colnames_de
    
    translate = \
    {'Stations_id':'station_id',
     'von_datum':'date_from',
     'bis_datum':'date_to',
     'Stationshoehe':'altitude',
     'geoBreite': 'latitude',
     'geoLaenge': 'longitude',
     'Stationsname':'name',
     'Bundesland':'state'}
    
    colnames_en = [translate[h] for h in colnames_de]
    
    # Skip the first two rows and set the column names.
    df = pd.read_fwf(txtfile,skiprows=2,names=colnames_en,encoding='iso8859_2', parse_dates=["date_from","date_to"],index_col = 0)
    
    # write csv
    df.to_csv(csvfile, sep = ";")
    return(df)

In [318]:
basename = os.path.splitext(station_fname)[0]
df_stations = station_desc_txt_to_csv(local_ftp_station_dir + station_fname, local_station_dir + basename + ".csv")


df_stations['date_from']=pd.to_datetime(df_stations['date_from'],infer_datetime_format=True) 
df_stations['date_to']=pd.to_datetime(df_stations['date_to'],infer_datetime_format=True)
df_stations.tail()

,date_from,date_to,altitude,latitude,longitude,name,state
station_id,,,,,,,
13713,2007-11-01,2012-01-08,386,51.0899,7.6289,Meinerzhagen-Redlendorf,Nordrhein-Westfalen
13777,2008-06-01,2012-01-09,110,52.2467,10.9592,Helmstedt-Emmerstedt,Niedersachsen
15000,2011-04-01,2022-03-02,231,50.7983,6.0244,Aachen-Orsbach,Nordrhein-Westfalen
15207,2013-11-01,2022-03-02,317,51.2835,9.3590,Schauenburg-Elgershausen,Hessen
15444,2014-09-01,2022-03-02,593,48.4418,9.9216,Ulm-Mähringen,Baden-Württemberg


### Select Stations Located in NRW from Station Description Dataframe

In [319]:
cloud_s_df=gpd.read_file('./data/generated/DWD/hourly/cloudiness/historical/cloud_stations_cologne.geojson', index_col='station_id')\
.set_crs(4326, allow_override=True).to_crs(5676).set_index('station_id')

In [320]:
cloud_s_df

,date_from,date_to,altitude,latitude,longitude,name,state,field_9,ID_0,ISO,...,NAME_1,ID_2,NAME_2,ID_3,NAME_3,TYPE_3,ENGTYPE_3,NL_NAME_3,VARNAME_3,geometry
station_id,,,,,,,,,,,,,,,,,,,,,
832,1979-01-02,1995-08-09,48,50.9833,6.9000,Butzweilerhof,Nordrhein-Westfalen,None,86,DEU,...,Nordrhein-Westfalen,29,Köln,289,Cologne Städte,Kreisfreie Städte,Urban district,None,Koln,MULTIPOINT Z (2563243.531 5650174.538 48.000)
2667,1949-01-01,2022-03-01,92,50.8646,7.1575,Köln-Bonn,Nordrhein-Westfalen,None,86,DEU,...,Nordrhein-Westfalen,29,Köln,289,Cologne Städte,Kreisfreie Städte,Urban district,None,Koln,MULTIPOINT Z (2581532.425 5637222.637 92.000)


In [321]:
station_ids_selected=list(cloud_s_df.index)

In [322]:


set(station_ids_selected).issubset(set(df_zips)) # we have some missing stations

False

In [323]:
df_zips.tail()

,name,ext,size,type
station_id,,,,
7412,stundenwerte_N_07412_akt.zip,.zip,45155,-
13674,stundenwerte_N_13674_akt.zip,.zip,45531,-
15000,stundenwerte_N_15000_akt.zip,.zip,45836,-
15207,stundenwerte_N_15207_akt.zip,.zip,45546,-
15444,stundenwerte_N_15444_akt.zip,.zip,45564,-


In [324]:
cloud_s_df.index.isin(df_zips.index)

array([False,  True])

### Download TS Data from FTP Server

Problem: Not all stations listed in the station description file are associated with a time series (zip file)! The stations in the description file and the set of stations whoch are TS data provided for (zip files) do not match perfectly.  

In [325]:
df_zips["name"][2667]

'stundenwerte_N_02667_akt.zip'

In [326]:
# Add the names of the zip files only to a list. 
local_zip_list = []

for station_id in station_ids_selected:
    try:
        fname = df_zips["name"][station_id]
        #print(fname)
        grabFile(ftp_dir + fname, local_ftp_ts_dir + fname)
        local_zip_list.append(fname)
    except:
        print("WARNING: TS file for key %d not found in FTP directory." % station_id)

### Concat the  Time Series dfs




In [327]:
# column names
#STATIONS_ID;MESS_DATUM;QN_9;TT_TU;RF_TU;eor

In [328]:
def temp_ts_to_df(fname):
    
    df = pd.read_csv(fname, delimiter=";", encoding="iso8859_2")
   
    
    # https://medium.com/@chaimgluck1/working-with-pandas-fixing-messy-column-names-42a54a6659cd

    # Column headers: remove leading blanks (strip), replace " " with "_", and convert to lower case.
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
    df['mess_datum']=pd.to_datetime(df['mess_datum'], format='%Y%m%d%H', utc=True, errors='ignore')
    
    begin_date=dt.datetime(2019, 8, 1, 0, 0, 0, tzinfo=dt.timezone.utc)
    df=df[df['mess_datum']>begin_date]
    return(df)

In [329]:
from zipfile import ZipFile

In [330]:
def ts_merge():
    # Very compact code.
    df = pd.DataFrame()
    for elt in local_zip_list:
        ffname = local_ftp_ts_dir + elt
        print("Zip archive: " + ffname)
        with ZipFile(ffname) as myzip:
            # read the time series data from the file starting with "produkt"
            prodfilename = [elt for elt in myzip.namelist() if elt.split("_")[0]=="produkt"][0] 
            print("Extract product file: %s" % prodfilename)
            print()
            with myzip.open(prodfilename) as myfile:
                dftmp = temp_ts_to_df(myfile)
                df = pd.concat([df, dftmp])

    
    return(df)

In [331]:
df_merged_ts = ts_merge()

Zip archive: data/original/DWD/hourly/cloudiness/recent/stundenwerte_N_02667_akt.zip
Extract product file: produkt_n_stunde_20200830_20220302_02667.txt



### Description

In [332]:
df_merged_ts.shape

(13193, 6)

In [333]:
df_merged_ts['mess_datum']=pd.to_datetime(df_merged_ts['mess_datum'], format='%Y%m%d%H', utc=True, errors='ignore')
df_merged_ts.head()

,stations_id,mess_datum,qn_8,v_n_i,v_n,eor
0,2667,2020-08-30 00:00:00+00:00,3,P,1,eor
1,2667,2020-08-30 01:00:00+00:00,3,P,1,eor
2,2667,2020-08-30 02:00:00+00:00,3,P,3,eor
3,2667,2020-08-30 03:00:00+00:00,3,P,2,eor
4,2667,2020-08-30 04:00:00+00:00,3,P,4,eor


In [334]:
df_stations.head(2)

,date_from,date_to,altitude,latitude,longitude,name,state
station_id,,,,,,,
3,1950-04-01,2011-04-01,202,50.7827,6.0941,Aachen,Nordrhein-Westfalen
5,1979-01-02,1995-06-30,190,50.8167,6.1833,Aachen-Merzbrück,Nordrhein-Westfalen


In [335]:
df_all=df_merged_ts.set_index('stations_id').join(df_stations, how='left')

In [336]:
df_merged_ts.to_csv(local_ts_merged_dir + "ts_merged.csv",sep=";")

In [337]:
df_stations.to_csv(local_ts_merged_dir + "df_stations.csv",sep=";")

In [338]:
df_all.head()

,mess_datum,qn_8,v_n_i,v_n,eor,date_from,date_to,altitude,latitude,longitude,name,state
2667,2020-08-30 00:00:00+00:00,3,P,1,eor,1949-01-01,2022-03-02,92,50.8646,7.1575,Köln-Bonn,Nordrhein-Westfalen
2667,2020-08-30 01:00:00+00:00,3,P,1,eor,1949-01-01,2022-03-02,92,50.8646,7.1575,Köln-Bonn,Nordrhein-Westfalen
2667,2020-08-30 02:00:00+00:00,3,P,3,eor,1949-01-01,2022-03-02,92,50.8646,7.1575,Köln-Bonn,Nordrhein-Westfalen
2667,2020-08-30 03:00:00+00:00,3,P,2,eor,1949-01-01,2022-03-02,92,50.8646,7.1575,Köln-Bonn,Nordrhein-Westfalen
2667,2020-08-30 04:00:00+00:00,3,P,4,eor,1949-01-01,2022-03-02,92,50.8646,7.1575,Köln-Bonn,Nordrhein-Westfalen


In [339]:
df_all.to_csv(local_ts_merged_dir + "df_all.csv",sep=";")

In [340]:
df_all.shape

(13193, 12)

In [341]:
df_all['mess_datum'].max(), df_all['mess_datum'].min()

(Timestamp('2022-03-02 23:00:00+0000', tz='UTC'),
 Timestamp('2020-08-30 00:00:00+0000', tz='UTC'))